```
This software is a part of GPU Ocean.

Copyright (C) 2019  SINTEF Digital

In this notebook we load the generated ensemble to look at the variance
of the observed drifters. This is done to come up with a good estimate for
the observation covariance matrix, R.

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>.
```

# Investigating the observation variance

In this notebook we load the generated ensemble to look at the variance
of the observed drifters. This is done to come up with a good estimate for
the observation covariance matrix, R.

## Set environment

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import animation, rc


import pycuda.driver as cuda
import os
import sys
import datetime

from importlib import reload
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '../../')))

#Set large figure sizes
rc('figure', figsize=(16.0, 12.0))
rc('animation', html='html5')

#Import our simulator
from SWESimulators import IPythonMagic, EnsembleFromFiles, EnsemblePlot, IEWPFOcean
from SWESimulators import CDKLM16, ParticleInfo, Observation

In [ ]:
%cuda_context_handler gpu_ctx

In [ ]:
#Create output directory for images
#imgdir = 'double_jet'
#filename_prefix = imgdir + "/" + datetime.datetime.now().strftime("%Y_%m_%d-%H_%M_%S") + "_"
#os.makedirs(imgdir, exist_ok=True)
#print("Saving images to " + imgdir)

# The data

The data used for this paper is available for download from [the GPU Ocean server](http://gpu-ocean.met.no:9000/gpu_ocean/DAPaper/).

Please start with downloading the files `double_jet_ensemble_init.zip` and `double_jet_truth.zip` to this folder, and unzip them so that the following lines of code passes:

In [ ]:
ensemble_init_path = os.path.abspath('double_jet_ensemble_init/')
assert len(os.listdir(ensemble_init_path)) == 102, "Ensemble init folder has wrong number of files"

truth_path = os.path.abspath('double_jet_truth/')
assert len(os.listdir(truth_path)) == 4, "Truth folder has wrong number of files"

### Define functions for plotting

In [ ]:
def imshow(im, interpolation="None", title=None, figsize=(4,4), interior=False):
    fig = plt.figure(figsize=figsize)
    
    if interior:
        im = plt.imshow(im[2:-2,2:-2], interpolation=interpolation, origin='lower')
    else:
        im = plt.imshow(im, interpolation=interpolation, origin='lower')
    
    plt.colorbar()
    if title is not None:
        plt.title(title)
        
def imshow3(eta, hu, hv, interpolation="None", title=None, figsize=(12,3), 
            interior=False, color_bar_from_zero=False):
    fig, axs = plt.subplots(1,3, figsize=figsize)
    
    eta_max = np.max(np.abs(eta))
    huv_max = max(np.max(np.abs(hu)), np.max(np.abs(hv)))
    eta_min = -eta_max
    huv_min = -huv_max
    if color_bar_from_zero:
        eta_min, huv_min = 0, 0
    
    if interior:
        eta_im = axs[0].imshow(eta[2:-2,2:-2], interpolation=interpolation, origin='lower', vmin=eta_min, vmax=eta_max)
    else:
        eta_im = axs[0].imshow(eta, interpolation=interpolation, origin='lower', vmin=eta_min, vmax=eta_max)
    axs[0].set_title("$\eta$")
    plt.colorbar(eta_im, ax=axs[0])
    
    if interior:
        hu_im = axs[1].imshow(hu[2:-2,2:-2], interpolation=interpolation, origin='lower', vmin=huv_min, vmax=huv_max)
    else:
        hu_im = axs[1].imshow(hu, interpolation=interpolation, origin='lower', vmin=huv_min, vmax=huv_max)
    axs[1].set_title("$hu$")
    plt.colorbar(hu_im, ax=axs[1])

    if interior:
        hv_im = axs[2].imshow(hv[2:-2,2:-2], interpolation=interpolation, origin='lower', vmin=huv_min, vmax=huv_max)
    else:
        hv_im = axs[2].imshow(hv, interpolation=interpolation, origin='lower', vmin=huv_min, vmax=huv_max)
    axs[2].set_title("$hv$")
    plt.colorbar(hv_im, ax=axs[2])

    if title is not None:
        plt.suptitle(title)
    plt.tight_layout()
    
def plotDrifters(observations, sim_reader, t, drifter_set=[]):
    drifter_positions = observations.get_drifter_position(t)

    fig = plt.figure(figsize=(7,7))
    ax = plt.subplot(111)
    nx, ny = sim_reader.get('nx'), sim_reader.get('ny')
    dx, dy = sim_reader.get('dx'), sim_reader.get('dy')

    emptyData =np.ma.masked_where(np.zeros((ny,nx)) > 1, np.zeros((ny,nx)))
    ax.imshow(emptyData, origin="lower", extent=[0, nx*dx, 0, ny*dy], cmap='binary')

    for i in range(drifter_positions.shape[0]):
        color = 'xkcd:pale cyan'
        if i in drifter_set:
            color = 'xkcd:tomato red'
        circ_end = matplotlib.patches.Circle((drifter_positions[i,0], drifter_positions[i,1]),
                                             3000, fill=True, zorder=10, color=color)
        ax.add_patch(circ_end)

    
def days_to_sec(days):
    return days*24*60*60

def truth_time_step(t):
    t = t - days_to_sec(3)
    return int(t/(60*60))

# Read initial conditions and mean and variance 

Reading files...

In [ ]:
if 'ensemble' in globals():
    ensemble.cleanUp()
    del ensemble
if 'iewpf' in globals():
    iewpf.cleanUp()
    del iewpf

reload(Observation)
reload(ParticleInfo)
reload(CDKLM16)    
reload(EnsembleFromFiles)
reload(IEWPFOcean)
ensemble = EnsembleFromFiles.EnsembleFromFiles(gpu_ctx, 20,
                                               ensemble_init_path, truth_path,
                                               15) #10) #0.2) #100)
ensemble.configureObservations(drifterSet=[4, 12, 20, 28, 36, 44, 52, 60])
iewpf = IEWPFOcean.IEWPFOcean(ensemble)

In [ ]:
#endtime = days_to_sec(3) + 5*60
#ensemble.stepToObservation(endtime, model_error_final_step=False, progress_info=True)
#EnsemblePlot.plotVelocityInfo(ensemble)
#print('hei')
#iewpf.iewpf_2stage(ensemble, perform_step=False)
#EnsemblePlot.plotVelocityInfo(ensemble)
#print('hei')


In [ ]:
%%time
################################################
#### This is good sketch for actual experiment!
################################################

info = ParticleInfo.ParticleInfo()
info.usePredefinedExtraCells()
endtime = days_to_sec(3)

for d in range(1): # days
    for h in range(24): # hours in a day
        #for h in range(4): # hours
        for i in range(12): # number of 5 min intervals per hour
            drifter_cells = ensemble.getDrifterCells()
            for m in range(5): # number of 1 min intervals within 5 min
                endtime += 60
                #print("\n--------------------------------------------------------")
                #print("ensemble.t, endtime, diff", (ensemble.t, endtime, endtime-ensemble.t))
                ensemble.stepToObservation(endtime, model_error_final_step=(m<4))
                if m == 4:
                    iewpf.iewpf_2stage(ensemble, perform_step=False)
                # TODO for each particle: Write hu, hv, eta under drifter positions to file 
                info.add_state_sample_from_sim(ensemble.particles[0], drifter_cells)
    
    # Write complete state to NetCDF
    # Dump ParticleInfo for this day
    print(str( (endtime - days_to_sec(3))/3600) + " hours into the simulation")
#iewpf.iewpf_2stage(ensemble, perform_step=False)



In [ ]:
filename = 'infoPlotTestbz2.bz2'
info.to_pickle(filename)

In [ ]:
infoFromFile = ParticleInfo.ParticleInfo()
infoFromFile.read_pickle(filename)

In [ ]:
print(info.get_num_drifters(), info.get_num_samples())
print(infoFromFile.get_num_drifters(), infoFromFile.get_num_samples())

times = infoFromFile.get_sample_times()
drifter_id = 0
eta = np.zeros_like(times)
hu  = np.zeros_like(times)
hv  = np.zeros_like(times)
ehu = np.zeros_like(times)
ehv = np.zeros_like(times)

for t in range(infoFromFile.get_num_samples()):
    sample = infoFromFile.get_state_samples(times[t])
    extra  = infoFromFile.get_extra_sample(times[t])
    eta[t] = sample[0,0]
    hu[t]  = sample[0,1]
    hv[t]  = sample[0,2]
    ehu[t] = extra[0,1]
    ehv[t] = extra[0,2]
    

#print(info.get_state_samples(times[20]))
    
times = times/(days_to_sec(1))
#plt.plot(times, eta)
plt.plot(times, hu/270)
plt.plot(times, hv/270)
plt.plot(times, ehu/270)
plt.plot(times, ehv/270)


In [ ]:
expected_run_time = 2*10*5
expected_storage = 1.3*10*100 + 100*10
print("expected_run_time", expected_run_time)
print("expected_storage", expected_storage)

In [ ]:
if 'sim' in globals():
    sim.cleanUp()
    del sim
reload(CDKLM16)

sim = CDKLM16.CDKLM16.fromfilename(gpu_ctx, "double_jet_ensemble_init/double_jet_case_00.nc", cont_write_netcdf=False)

for i in range(2):
    start_t = sim.t
    end_t = start_t + 60
    sim.dataAssimilationStep(end_t)

print(end_t, sim.t, end_t - sim.t)



In [ ]:
ensemble.getNumDrifters()

In [ ]:
ensemble.stepToObservation(days_to_sec(3)+15*60)
drifter_positions = ensemble.observeTrueDrifters()
print(drifter_positions)
drifter_positions[:,0] = np.floor(drifter_positions[:,0]/ensemble.getDx())#.astype(np.int32)
drifter_positions[:,1] = np.floor(drifter_positions[:,1]/ensemble.getDy())#.astype(np.int32)
drifter_positions = drifter_positions.astype(np.int32)
#drifter_positions[:,0] = np.int(np.floor(drifter_positions[:,0]/ensemble.getDx()))
#drifter_positions[:,1] = np.int(np.floor(drifter_positions[:,1]/ensemble.getDy()))
print(drifter_positions)
print(drifter_positions.dtype)
print(ensemble.getDrifterCells())


#id_x = np.int(np.floor(trueState[d,0]/self.dx))
#id_y = np.int(np.floor(trueState[d,1]/self.dy))

In [ ]:
print(ensemble.observations.get_drifter_position(ensemble.t))
print(ensemble.observeTrueDrifters() - ensemble.observations.get_drifter_position(ensemble.t))


In [ ]:

eta, hu, hv = ensemble.downloadParticleOceanState(0)
for i in drifter_positions:
    print(eta[i[1],i[0]], hu[i[1], i[0]], hv[i[1], i[0]])
drifter_positions.shape
print('----------')
#etas = eta[drifter_positions[:,1], drifter_positions[:,0]]
#hus  =  hu[drifter_positions[:,1], drifter_positions[:,0]]
#hvs  =  hv[drifter_positions[:,1], drifter_positions[:,0]]

state_sample = np.zeros((drifter_positions.shape[0], 3))
#state_sample[:,0] = etas
#state_sample[:,1] = hus
#state_sample[:,2] = hvs
state_sample[:,0] = eta[drifter_positions[:,1], drifter_positions[:,0]]
state_sample[:,1] = hu[drifter_positions[:,1], drifter_positions[:,0]]
state_sample[:,2] = hv[drifter_positions[:,1], drifter_positions[:,0]]


print(state_sample)


In [ ]:
ensemble.particles[0].model_time_step, m, endtime

In [ ]:
EnsemblePlot.plotVelocityInfo(ensemble)
print('hei')

In [ ]:
EnsemblePlot.plotVelocityInfo(ensemble)
print('hei')

In [ ]:
for i in range(100):
    iewpf.iewpf_2stage(ensemble, perform_step=False)
EnsemblePlot.plotVelocityInfo(ensemble)
print('hei')

In [ ]:
just_after_endtime = endtime + 300
ensemble.stepToObservation(just_after_endtime, model_error_final_step=False, progress_info=True)
EnsemblePlot.plotVelocityInfo(ensemble)
print('hei')

In [ ]:
15/np.sqrt(ensemble.mean_depth)

# Step far ahead

In [ ]:
endtime = days_to_sec(4)
ensemble.stepToObservation(endtime, model_error_final_step=False, progress_info=True)

In [ ]:
EnsemblePlot.plotVelocityInfo(ensemble)
print('hei')
iewpf.iewpf_2stage(ensemble, perform_step=False)
EnsemblePlot.plotVelocityInfo(ensemble)
print('hei')


In [ ]:
obs_times = ensemble.observations.get_observation_times()
obs_times[1] - obs_times[0], 300/60

In [ ]:
observations = ensemble.observeParticles()
print(observations.shape)

In [ ]:
print(np.var(observations, axis=0)/ensemble.mean_depth**2)

In [ ]:
print(np.sqrt(np.var(observations/ensemble.mean_depth, axis=0)))


In [ ]:
innovations = ensemble.getInnovations()
print(np.sqrt(np.var(innovations/ensemble.mean_depth, axis=0)))
print(np.mean(innovations/ensemble.mean_depth, axis=0))

